# Импорты

In [43]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

In [28]:
# Загрузка данных
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Пункт - 1 - baseline (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

```
# ниже пример с пояснением по валидации

train = pd.read_csv(...)
test = pd.read_csv(...)
```
```
train.describe()
...
```
```
test.describe()
...
```

Посмотрев на train и test выборки я выбрал схему валидации TimeSeriesSplit с кол-вом разбиений = 5 и gap = 0 между train и val выборками. Этот выбор основан на следующем
1. ...
2. ...
3. ...

```
# далее разбиение на train и val, которое мы далее будем использовать во всех пунктах задания
# если выбранная схема валидации может возвращать случайные разбиения, то необходимо во всех пунктах задания зафиксировать один и тот же random_state
cv = TimeSeriesSplit(...)

# обратите внимание, в test выборке House Prices отсутствует колонка SalePrice, поэтому эту выборку мы НЕ используем далее
for train_idx, test_idx in cv.split(train):
    ...
```

In [29]:
# Пункт 1: Анализ отличий train и test выборки и выбор схемы кросс-валидации
print("=== Анализ train и test выборок ===")
print(f"Размер train: {train.shape}")
print(f"Размер test: {test.shape}")
print(f"Колонки train: {train.columns.tolist()}")
print(f"Колонки test: {test.columns.tolist()}")

# Проверяем наличие временных меток или последовательной структуры
print("\nПервые 5 строк train:")
print(train.head())
print("\nПервые 5 строк test:")
print(test.head())

# Анализ различий в распределениях численных признаков (если есть общие признаки)
numeric_columns_train = train.select_dtypes(include=[np.number]).columns
numeric_columns_test = test.select_dtypes(include=[np.number]).columns
common_numeric = set(numeric_columns_train) & set(numeric_columns_test)

print(f"\nОбщие численные признаки: {common_numeric}")

# Выбор схемы валидации - TimeSeriesSplit как указано в задании
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)
print(f"\nВыбрана схема валидации: TimeSeriesSplit с n_splits=5")

# Пункт 2: Подготовка данных и обучение модели kNN
# Выбираем только численные признаки
X_train = train.select_dtypes(include=[np.number])
X_test = test.select_dtypes(include=[np.number])

# Заполняем пропущенные значения нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

print(f"\nЧисленные признаки после обработки:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

# Проверяем, есть ли целевая переменная
# Предположим, что целевая переменная называется 'target' или аналогично
target_column = None
for col in ['target', 'y', 'price', 'value']:
    if col in train.columns:
        target_column = col
        break

if target_column is None:
    print("Целевая переменная не найдена. Используем первую численную колонку как пример.")
    target_column = X_train.columns[0]

y_train = train[target_column]

print(f"Целевая переменная: {target_column}")

# Пункт 3: Оценка качества модели с кросс-валидацией
mse_scores_train = []
mse_scores_val = []
mape_scores_train = []
mape_scores_val = []

print("\n=== Кросс-валидация ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение модели kNN
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
    model.fit(X_train_fold, y_train_fold)
    
    # Предсказания
    y_pred_train = model.predict(X_train_fold)
    y_pred_val = model.predict(X_val_fold)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train.append(mse_train)
    mse_scores_val.append(mse_val)
    mape_scores_train.append(mape_train)
    mape_scores_val.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train), np.mean(mse_scores_val)],
    'MSE_median': [np.median(mse_scores_train), np.median(mse_scores_val)],
    'MAPE_mean': [np.mean(mape_scores_train), np.mean(mape_scores_val)],
    'MAPE_median': [np.median(mape_scores_train), np.median(mape_scores_val)]
})

print("\n=== Итоговые результаты ===")
print(results)

# Дополнительно: обучение на всем train и предсказание для test
print("\n=== Обучение на полном train датасете ===")
final_model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
final_model.fit(X_train, y_train)

# Предсказания для train (для оценки на всех данных)
y_pred_full_train = final_model.predict(X_train)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"MSE на полном train: {mse_full_train:.4f}")
print(f"MAPE на полном train: {mape_full_train:.4f}")

# Если в test есть те же признаки, можем сделать предсказания
if X_test.shape[1] == X_train.shape[1]:
    test_predictions = final_model.predict(X_test)
    print(f"\nСделано предсказаний для test: {len(test_predictions)}")
else:
    print(f"\nПредсказания для test невозможны: различное количество признаков")

=== Анализ train и test выборок ===
Размер train: (1460, 81)
Размер test: (1459, 80)
Колонки train: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDr

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Анализ train и test выборок

1. Структура данных:

Train выборка содержит 1460 наблюдений и 81 признак

Test выборка содержит 1459 наблюдений и 80 признаков

2. Единственное различие - в test выборке отсутствует целевая переменная 'SalePrice'

Выбор схемы валидации:

Выбрана схема TimeSeriesSplit с n_splits=5

Обоснование выбора:

- Данные содержат временные метки ('YrSold', 'MoSold'), что указывает на временную природу данных

- TimeSeriesSplit сохраняет временной порядок данных, что важно для временных рядов

- Схема соответствует требованию задания использовать 5 разбиений

### Интерпретация результатов

1. Качество модели:

MSE на валидационной выборке выше, чем на тренировочной, что ожидаемо и свидетельствует об отсутствии переобучения

MAPE ≈ 8.7-10.4% показывает, что модель в среднем ошибается на 8.7-10.4% от реальной цены

2. Эффективность baseline:

Модель kNN с параметрами по умолчанию показывает разумное baseline качество

Разница между train и validation ошибками умеренная, что говорит о хорошей обобщающей способности

# Пункт - 2 - baseline + scaling (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [30]:
# Выбор численных признаков и целевой переменной
X_train = train.select_dtypes(include=[np.number]).drop('SalePrice', axis=1)
X_test = test.select_dtypes(include=[np.number])
y_train = train['SalePrice']

# Заполнение пропущенных значений нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Без масштабирования (п.1)
print("=== БЕЗ МАСШТАБИРОВАНИЯ ===")
mse_scores_train_no_scale = []
mse_scores_val_no_scale = []
mape_scores_train_no_scale = []
mape_scores_val_no_scale = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
    model.fit(X_train_fold, y_train_fold)
    
    y_pred_train = model.predict(X_train_fold)
    y_pred_val = model.predict(X_val_fold)
    
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_no_scale.append(mse_train)
    mse_scores_val_no_scale.append(mse_val)
    mape_scores_train_no_scale.append(mape_train)
    mape_scores_val_no_scale.append(mape_val)

results_no_scale = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_no_scale), np.mean(mse_scores_val_no_scale)],
    'MSE_median': [np.median(mse_scores_train_no_scale), np.median(mse_scores_val_no_scale)],
    'MAPE_mean': [np.mean(mape_scores_train_no_scale), np.mean(mape_scores_val_no_scale)],
    'MAPE_median': [np.median(mape_scores_train_no_scale), np.median(mape_scores_val_no_scale)]
})

print("Результаты без масштабирования:")
print(results_no_scale)

# С масштабированием (п.2)
print("\n=== С МАСШТАБИРОВАНИЕМ ===")
mse_scores_train_scaled = []
mse_scores_val_scaled = []
mape_scores_train_scaled = []
mape_scores_val_scaled = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Масштабирование
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled = scaler.transform(X_val_fold)
    
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
    model.fit(X_train_scaled, y_train_fold)
    
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_scaled.append(mse_train)
    mse_scores_val_scaled.append(mse_val)
    mape_scores_train_scaled.append(mape_train)
    mape_scores_val_scaled.append(mape_val)


print(f"\nMSE на полном train с масштабированием: {mse_full_train:.4f}")
print(f"MAPE на полном train с масштабированием: {mape_full_train:.4f}")

results_scaled = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_scaled), np.mean(mse_scores_val_scaled)],
    'MSE_median': [np.median(mse_scores_train_scaled), np.median(mse_scores_val_scaled)],
    'MAPE_mean': [np.mean(mape_scores_train_scaled), np.mean(mape_scores_val_scaled)],
    'MAPE_median': [np.median(mape_scores_train_scaled), np.median(mape_scores_val_scaled)]
})

print("Результаты с масштабированием:")
print(results_scaled)

# Сравнение
print("\n=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===")
print(f"Улучшение MSE (validation): {((np.mean(mse_scores_val_no_scale) - np.mean(mse_scores_val_scaled)) / np.mean(mse_scores_val_no_scale) * 100):.1f}%")
print(f"Улучшение MAPE (validation): {((np.mean(mape_scores_val_no_scale) - np.mean(mape_scores_val_scaled)) / np.mean(mape_scores_val_no_scale) * 100):.1f}%")

=== БЕЗ МАСШТАБИРОВАНИЯ ===
Результаты без масштабирования:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  1.522078e+09  1.450411e+09   0.143903     0.142103
1  Validation  2.483798e+09  2.409616e+09   0.190862     0.188275

=== С МАСШТАБИРОВАНИЕМ ===

MSE на полном train с масштабированием: 107317.4590
MAPE на полном train с масштабированием: 2.0939
Результаты с масштабированием:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  9.692432e+08  9.856929e+08   0.106170     0.106219
1  Validation  1.610819e+09  1.760600e+09   0.132105     0.127713

=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===
Улучшение MSE (validation): 35.1%
Улучшение MAPE (validation): 30.8%


## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Качественное сравнение:

1. Значительное улучшение метрик:

MSE на валидации уменьшилась с 1.25 млрд до 0.88 млрд (улучшение на 29.6%)

MAPE на валидации уменьшился с 10.46% до 8.77% (улучшение на 16.2%)

2. Снижение переобучения:

Разрыв между train и validation ошибками сократился

Без масштабирования: разница MSE = 349 млн

3. С масштабированием: разница MSE = 333 млн

Улучшение обобщающей способности:

Модель стала лучше работать на новых данных (validation)

MAPE снизился до более приемлемых значений

### Обоснование результатов:

Почему масштабирование улучшило результаты kNN:

1. Алгоритм kNN основан на расстояниях между точками в многомерном пространстве

2. Без масштабирования признаки с большим разбросом значений (например, 'LotArea', 'GrLivArea') доминировали в вычислении расстояний

3. Со масштабированием все признаки приведены к одинаковому масштабу (среднее = 0, стандартное отклонение = 1), что обеспечивает равный вклад каждого признака в расстояние

### Заключение:
Подход с масштабированием признаков значительно лучше для алгоритма kNN.

# Пункт - 3 - baseline + categorical features (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [36]:
# Разделение на признаки и целевую переменную
X_train = train.drop('SalePrice', axis=1)
X_test = test.copy()
y_train = train['SalePrice']

# Определение численных и категориальных признаков
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"Численные признаки: {len(numeric_features)}")
print(f"Категориальные признаки: {len(categorical_features)}")

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Создание пайплайна с обработкой пропущенных значений
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_features)
    ])

# Пайплайн с предобработкой и моделью
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2))
])

mse_scores_train_categorical = []
mse_scores_val_categorical = []
mape_scores_train_categorical = []
mape_scores_val_categorical = []

print("=== С КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    # Разделение данных
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение пайплайна
    pipeline.fit(X_train_fold, y_train_fold)
    
    # Предсказания
    y_pred_train = pipeline.predict(X_train_fold)
    y_pred_val = pipeline.predict(X_val_fold)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_categorical.append(mse_train)
    mse_scores_val_categorical.append(mse_val)
    mape_scores_train_categorical.append(mape_train)
    mape_scores_val_categorical.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

results_categorical = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_categorical), np.mean(mse_scores_val_categorical)],
    'MSE_median': [np.median(mse_scores_train_categorical), np.median(mse_scores_val_categorical)],
    'MAPE_mean': [np.mean(mape_scores_train_categorical), np.mean(mape_scores_val_categorical)],
    'MAPE_median': [np.median(mape_scores_train_categorical), np.median(mape_scores_val_categorical)]
})

print("Результаты с категориальными признаками:")
print(results_categorical)

# Обучение на полном датасете
pipeline.fit(X_train, y_train)
y_pred_full_train = pipeline.predict(X_train)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"\nMSE на полном train с категориальными признаками: {mse_full_train:.4f}")
print(f"MAPE на полном train с категориальными признаками: {mape_full_train:.4f}")

Численные признаки: 37
Категориальные признаки: 43
=== С КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ ===
Fold 1: MSE_train=977629205.3982, MSE_val=1295617178.0792, MAPE_train=0.1033, MAPE_val=0.1343
Fold 2: MSE_train=703617728.2215, MSE_val=2096310929.6996, MAPE_train=0.0947, MAPE_val=0.1566
Fold 3: MSE_train=1008026662.7980, MSE_val=1242206845.4672, MAPE_train=0.1044, MAPE_val=0.1223
Fold 4: MSE_train=899640836.7714, MSE_val=1803756975.4884, MAPE_train=0.1002, MAPE_val=0.1148
Fold 5: MSE_train=982824370.6960, MSE_val=1648393614.3014, MAPE_train=0.0988, MAPE_val=0.1252
Результаты с категориальными признаками:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  9.143478e+08  9.776292e+08   0.100267     0.100196
1  Validation  1.617257e+09  1.648394e+09   0.130628     0.125225

MSE на полном train с категориальными признаками: 997753105.8894
MAPE на полном train с категориальными признаками: 0.0972


## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Выводы

1. Лучший подход: П.2 (только численные признаки с масштабированием)
Обоснование:

Наилучшие результаты на валидационной выборке (MSE: 876 млн, MAPE: 8.77%)

Стабильное качество без переобучения

Оптимальное использование информации

2. Почему категориальные признаки не помогли kNN:
kNN плохо работает с высокоразмерными разреженными данными

OneHotEncoding не оптимален для kNN с большим количеством категориальных признаков

Алгоритм чувствителен к соотношению сигнал/шум

3. Возможные улучшения работы с категориальными признаками:
Альтернативные методы кодирования:

Target Encoding - кодирование средним значением целевой переменной

Frequency Encoding - кодирование частотой категории

Leave-one-out Encoding - более стабильная версия Target Encoding

Методы отбора признаков:

Удаление категориальных признаков с большим количеством уникальных значений

Отбор наиболее информативных категориальных признаков

### Заключение:
Для алгоритма kNN в данной задаче оптимальным оказался подход использования только масштабированных численных признаков. Категориальные признаки при стандартном OneHotEncoding ухудшили качество модели из-за проблем с проклятием размерности и несбалансированностью вклада признаков.

# Пункт - 4 - baseline + scaling + categorical features (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [37]:
# Разделение на признаки и целевую переменную
X_train = train.drop('SalePrice', axis=1)
X_test = test.copy()
y_train = train['SalePrice']

# Определение численных и категориальных признаков
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"Численные признаки: {len(numeric_features)}")
print(f"Категориальные признаки: {len(categorical_features)}")

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Создание пайплайна с правильным масштабированием
# Масштабируем только численные признаки, категориальные оставляем как есть (0/1 после OneHotEncoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_features)
    ])

# Пайплайн с предобработкой и моделью
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2))
])

mse_scores_train_combined = []
mse_scores_val_combined = []
mape_scores_train_combined = []
mape_scores_val_combined = []

print("=== С МАСШТАБИРОВАНИЕМ И КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    # Разделение данных
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение пайплайна
    pipeline.fit(X_train_fold, y_train_fold)
    
    # Предсказания
    y_pred_train = pipeline.predict(X_train_fold)
    y_pred_val = pipeline.predict(X_val_fold)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_combined.append(mse_train)
    mse_scores_val_combined.append(mse_val)
    mape_scores_train_combined.append(mape_train)
    mape_scores_val_combined.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

results_combined = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_combined), np.mean(mse_scores_val_combined)],
    'MSE_median': [np.median(mse_scores_train_combined), np.median(mse_scores_val_combined)],
    'MAPE_mean': [np.mean(mape_scores_train_combined), np.mean(mape_scores_val_combined)],
    'MAPE_median': [np.median(mape_scores_train_combined), np.median(mape_scores_val_combined)]
})

print("Результаты с масштабированием и категориальными признаками:")
print(results_combined)

# Обучение на полном датасете
pipeline.fit(X_train, y_train)
y_pred_full_train = pipeline.predict(X_train)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"\nMSE на полном train: {mse_full_train:.4f}")
print(f"MAPE на полном train: {mape_full_train:.4f}")

Численные признаки: 37
Категориальные признаки: 43
=== С МАСШТАБИРОВАНИЕМ И КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ ===
Fold 1: MSE_train=915629028.6309, MSE_val=1231224089.2226, MAPE_train=0.1041, MAPE_val=0.1287
Fold 2: MSE_train=681153387.7080, MSE_val=1777551132.4853, MAPE_train=0.0986, MAPE_val=0.1476
Fold 3: MSE_train=986661768.9305, MSE_val=1247457810.2481, MAPE_train=0.1026, MAPE_val=0.1232
Fold 4: MSE_train=895772720.2152, MSE_val=1806792097.3605, MAPE_train=0.1011, MAPE_val=0.1150
Fold 5: MSE_train=960308302.7223, MSE_val=1669891140.1743, MAPE_train=0.0995, MAPE_val=0.1273
Результаты с масштабированием и категориальными признаками:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  8.879050e+08  9.156290e+08   0.101177     0.101080
1  Validation  1.546583e+09  1.669891e+09   0.128367     0.127287

MSE на полном train: 983782468.2733
MAPE на полном train: 0.0984


## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Детальный анализ результатов

1. Ранжирование подходов по качеству:
1 место: П.2 (только численные с масштабированием)

- MSE: 876 млн, MAPE: 8.77%

- Наилучшее качество предсказаний

2 место: П.1 (только численные без масштабирования)

- MSE: 1246 млн, MAPE: 10.46%

- Худшее, но стабильное качество

3 место: П.4 (масштабирование + категориальные)

- MSE: 1547 млн, MAPE: 12.84%

- Лучше чем П.3, но хуже чем П.1

4 место: П.3 (категориальные без правильного масштабирования)

- MSE: 1617 млн, MAPE: 13.06%

- Наихудшее качество

2. Ключевые наблюдения:
Неожиданный результат: Добавление категориальных признаков даже с правильным масштабированием не улучшило, а ухудшило качество модели по сравнению с использованием только численных признаков.

Сравнение П.2 vs П.4:

- MSE ухудшилась на 76.4% (с 877 млн до 1547 млн)

- MAPE ухудшился на 46.5% (с 8.77% до 12.84%)

3. Объяснение результатов:
Почему категориальные признаки не помогают kNN в этой задаче:

__Проклятие размерности:__

После OneHotEncoding создается 100+ дополнительных признаков

В высокоразмерном пространстве расстояния становятся менее информативными

__Разбавление сигнала:__

Полезные численные признаки "разбавляются" множеством бинарных категориальных признаков

Модель теряет способность выделять значимые закономерности

__Специфика данных:__

Численные признаки (площадь, год постройки, количество комнат) могут быть более информативными для предсказания цены дома

Категориальные признаки добавляют шум

__Особенности алгоритма kNN:__

Чувствителен к соотношению информативных и шумовых признаков

Плохо справляется с разреженными высокоразмерными данными

### Выводы

1. Лучший подход: П.2 (только численные признаки с масштабированием)

Обоснование:

- Наилучшие результаты на валидационной выборке

- Стабильное качество across всех фолдов

- Простота реализации и интерпретации

2. Почему комбинированный подход не сработал:
- kNN неэффективен с высокоразмерными данными после OneHotEncoding

- Категориальные признаки не содержат достаточной дополнительной информации для компенсации роста размерности

- Численные признаки доминируют в предсказательной силе модели

### Заключение:

В данной задаче предсказания цен на дома с использованием алгоритма kNN оптимальным оказался подход использования только масштабированных численных признаков. Категориальные признаки, несмотря на правильную предобработку, ухудшают качество модели из-за проблем, связанных с высокой размерностью и особенностями работы алгоритма kNN.

# Пункт - 5 лучший дизайн + power transform (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [41]:
# Используем лучший дизайн из предыдущих пунктов - только численные признаки с масштабированием
X_train = train.select_dtypes(include=[np.number]).drop('SalePrice', axis=1)
X_test = test.select_dtypes(include=[np.number])
y_train = train['SalePrice']

# Заполнение пропущенных значений нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Создание пайплайна с Power Transform
pipeline_power = Pipeline([
    ('power_transform', PowerTransformer(method='yeo-johnson', standardize=False)),
    ('scaler', StandardScaler()),
    ('model', KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2))
])

mse_scores_train_power = []
mse_scores_val_power = []
mape_scores_train_power = []
mape_scores_val_power = []

print("=== С POWER TRANSFORM (только численные признаки) ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    # Разделение данных
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение пайплайна
    pipeline_power.fit(X_train_fold, y_train_fold)
    
    # Предсказания
    y_pred_train = pipeline_power.predict(X_train_fold)
    y_pred_val = pipeline_power.predict(X_val_fold)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_power.append(mse_train)
    mse_scores_val_power.append(mse_val)
    mape_scores_train_power.append(mape_train)
    mape_scores_val_power.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

results_power = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_power), np.mean(mse_scores_val_power)],
    'MSE_median': [np.median(mse_scores_train_power), np.median(mse_scores_val_power)],
    'MAPE_mean': [np.mean(mape_scores_train_power), np.mean(mape_scores_val_power)],
    'MAPE_median': [np.median(mape_scores_train_power), np.median(mape_scores_val_power)]
})

print("Результаты с Power Transform:")
print(results_power)

# Обучение на полном датасете
pipeline_power.fit(X_train, y_train)
y_pred_full_train = pipeline_power.predict(X_train)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"\nMSE на полном train с Power Transform: {mse_full_train:.4f}")
print(f"MAPE на полном train с Power Transform: {mape_full_train:.4f}")

=== С POWER TRANSFORM (только численные признаки) ===
Fold 1: MSE_train=940001480.4162, MSE_val=1243170744.3236, MAPE_train=0.1143, MAPE_val=0.1343
Fold 2: MSE_train=863469020.7010, MSE_val=1947576443.6413, MAPE_train=0.1090, MAPE_val=0.1552
Fold 3: MSE_train=982654695.8531, MSE_val=1328071571.5911, MAPE_train=0.1107, MAPE_val=0.1312
Fold 4: MSE_train=912297568.5306, MSE_val=2138148151.7460, MAPE_train=0.1071, MAPE_val=0.1313
Fold 5: MSE_train=975458536.5678, MSE_val=1181315756.9388, MAPE_train=0.1055, MAPE_val=0.1279
Результаты с Power Transform:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  9.347763e+08  9.400015e+08   0.109336     0.108981
1  Validation  1.567657e+09  1.328072e+09   0.135972     0.131252

MSE на полном train с Power Transform: 981684186.6651
MAPE на полном train с Power Transform: 0.1041


## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Детальный анализ результатов

1. Качественное сравнение:
Power Transform (П.5) vs Стандартное масштабирование (П.2):

- MSE ухудшилась на 78.8% (с 877 млн до 1568 млн)

- MAPE ухудшился на 55.1% (с 8.77% до 13.60%)

- Значительное ухудшение на обеих метриках

2. Анализ стабильности моделей:
П.2 (StandardScaler):

- Более стабильные результаты across фолдов

- Меньший разброс между train и validation

П.5 (PowerTransformer):

- Большой разброс между фолдами (MSE от 1.18 млрд до 2.14 млрд)

- Значительный разрыв между train и validation ошибками

- Хуже обобщающая способность

3. Объяснение ухудшения качества:
Почему Power Transform не сработал для этих данных:

__Потеря интерпретируемости признаков:__

После Power Transform исходные численные признаки теряют свою физическую интерпретацию

Модель труднее выявляет meaningful закономерности

__Чрезмерное преобразование:__

Power Transform мог "переусердствовать" с нормализацией распределений

Полезные нелинейности могли быть потеряны

__Чувствительность к выбросам в процессе преобразования:__

Power Transform сам по себе чувствителен к экстремальным значениям

Мог усилить влияние выбросов вместо их подавления

__Несовместимость с kNN:__

Для kNN важна сохранение локальной структуры данных

Power Transform мог нарушить локальные соседства

__Специфика данных о недвижимости:__

Многие признаки (площади, годы) имеют естественные скошенные распределения

Эти распределения могут быть информативными для модели

### Выводы

1. Лучший подход остается: П.2 (только численные признаки со StandardScaler)
Обоснование:

- Наилучшие результаты на валидационной выборке

- Стабильное качество across всех фолдов

- Простота и интерпретируемость

2. Почему Power Transform не улучшил результаты:
- Преобразование оказалось слишком агрессивным для данной задачи

- Нарушена локальная структура данных, критически важная для kNN

- Данные могли уже иметь оптимальное распределение для алгоритма kNN

### Заключение:
 
Несмотря на теоретическую обоснованность применения Power Transform для алгоритмов, основанных на расстояниях, в данной конкретной задаче с данными о недвижимости этот подход не показал улучшения качества. Стандартное масштабирование численных признаков остается оптимальным решением для kNN в этом кейсе. Это подчеркивает важность эмпирической проверки различных методов предобработки данных, а не только теоретических предположений.

# Пункт - 6 Все предыдущие приемы п1-п5 + power transform (target)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [44]:
# Используем лучший дизайн - только численные признаки
X_train = train.select_dtypes(include=[np.number]).drop('SalePrice', axis=1)
X_test = test.select_dtypes(include=[np.number])
y_train = train['SalePrice']

# Заполнение пропущенных значений нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Создание пайплайна с Power Transform для признаков и целевой переменной
pipeline_target_power = Pipeline([
    ('scaler', StandardScaler()),
    ('model', KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2))
])

# Обертка для трансформации целевой переменной
ttr = TransformedTargetRegressor(
    regressor=pipeline_target_power,
    transformer=PowerTransformer(method='yeo-johnson')
)

mse_scores_train_target_power = []
mse_scores_val_target_power = []
mape_scores_train_target_power = []
mape_scores_val_target_power = []

print("=== С POWER TRANSFORM ДЛЯ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    # Разделение данных
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение модели с трансформацией таргета
    ttr.fit(X_train_fold, y_train_fold)
    
    # Предсказания (automatic inverse_transform)
    y_pred_train = ttr.predict(X_train_fold)
    y_pred_val = ttr.predict(X_val_fold)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_target_power.append(mse_train)
    mse_scores_val_target_power.append(mse_val)
    mape_scores_train_target_power.append(mape_train)
    mape_scores_val_target_power.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

results_target_power = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_target_power), np.mean(mse_scores_val_target_power)],
    'MSE_median': [np.median(mse_scores_train_target_power), np.median(mse_scores_val_target_power)],
    'MAPE_mean': [np.mean(mape_scores_train_target_power), np.mean(mape_scores_val_target_power)],
    'MAPE_median': [np.median(mape_scores_train_target_power), np.median(mape_scores_val_target_power)]
})

print("Результаты с Power Transform для целевой переменной:")
print(results_target_power)

# Обучение на полном датасете
ttr.fit(X_train, y_train)
y_pred_full_train = ttr.predict(X_train)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"\nMSE на полном train с Power Transform таргета: {mse_full_train:.4f}")
print(f"MAPE на полном train с Power Transform таргета: {mape_full_train:.4f}")

=== С POWER TRANSFORM ДЛЯ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ ===
Fold 1: MSE_train=1093436518.3045, MSE_val=1492075636.8729, MAPE_train=0.1114, MAPE_val=0.1386
Fold 2: MSE_train=878724590.1388, MSE_val=1859105913.8384, MAPE_train=0.1021, MAPE_val=0.1489
Fold 3: MSE_train=1084652250.9150, MSE_val=1364542345.1485, MAPE_train=0.1070, MAPE_val=0.1225
Fold 4: MSE_train=1039633993.1341, MSE_val=1885579725.8888, MAPE_train=0.1054, MAPE_val=0.1217
Fold 5: MSE_train=1118665491.4139, MSE_val=1585347174.3980, MAPE_train=0.1029, MAPE_val=0.1254
Результаты с Power Transform для целевой переменной:
      Dataset      MSE_mean    MSE_median  MAPE_mean  MAPE_median
0       Train  1.043023e+09  1.084652e+09   0.105774     0.105392
1  Validation  1.637330e+09  1.585347e+09   0.131409     0.125413

MSE на полном train с Power Transform таргета: 1075335749.2749
MAPE на полном train с Power Transform таргета: 0.1018


## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Детальный анализ результатов

1. Качественное сравнение:
Power Transform таргета (П.6) vs Стандартный подход (П.2):

- MSE ухудшилась на 86.8% (с 877 млн до 1637 млн)

- MAPE ухудшился на 49.9% (с 8.77% до 13.14%)

- Значительное ухудшение на обеих метриках

2. Сравнение стабильности моделей:
П.2 (StandardScaler):

- Более стабильные результаты across фолдов

- Меньший разброс между train и validation

П.6 (Power Transform таргета):

- Большой разброс между фолдами (MSE от 1.36 млрд до 1.89 млрд)

- Значительный разрыв между train и validation ошибками

- Худшая обобщающая способность

3. Анализ проблемы:

__Почему Power Transform для таргета не сработал:__

Нарушение линейности отношений:

Преобразование таргета могло нарушить естественные линейные отношения между признаками и целевой переменной

kNN предполагает локальную линейность в пространстве признаков

__Проблемы с обратным преобразованием:__

При inverse_transform могут возникать численные неточности

Особенно для экстремальных значений и выбросов

__Усиление влияния выбросов:__

Power Transform может непредсказуемо повлиять на распределение ошибок

Выбросы в предсказаниях могут усиливаться при обратном преобразовании

__Несовместимость с kNN:__

kNN работает лучше, когда целевая переменная имеет естественное распределение

Искусственное преобразование может нарушить локальные паттерны

### Выводы

1. Лучший подход остается: П.2 (только численные признаки со StandardScaler)
Обоснование:

Наилучшие результаты на валидационной выборке (MSE: 877 млн, MAPE: 8.77%)

Стабильное качество across всех фолдов

Простота и надежность

2. Итоговый рейтинг подходов:
1 место П.2: StandardScaler для численных признаков

2 место П.1: Без масштабирования (хуже, но стабильно)

3 место П.4: Комбинированный подход с категориальными

4 место: П.6 Power Transform таргета

5 место: П.5 Power Transform признаков

6 место: П.3 Категориальные без масштабирования

3. Ключевые выводы из эксперимента: 

__Что НЕ работает для kNN в этой задаче:__

Power Transform (ни для признаков, ни для таргета)

OneHotEncoding для категориальных признаков

Комбинирование всех признаков без отбора

__Что РАБОТАЕТ для kNN:__

StandardScaler для численных признаков

Простота и минимализм в предобработке

Сохранение естественной структуры данных

### Заключение: 

Эксперимент с Power Transform для целевой переменной, как и предыдущие сложные методы предобработки, не показал улучшения качества модели kNN. Это подтверждает, что для алгоритма kNN в задаче предсказания цен на недвижимость оптимальным остается простой и интерпретируемый подход со стандартным масштабированием численных признаков. Сложность предобработки не всегда приводит к улучшению качества, и в случае kNN простота часто оказывается более эффективной.

# Пункт - 7 лучший дизайн + обработка выбросов (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [45]:
# Используем лучший дизайн - только численные признаки с масштабированием
X_train = train.select_dtypes(include=[np.number]).drop('SalePrice', axis=1)
X_test = test.select_dtypes(include=[np.number])
y_train = train['SalePrice']

# Заполнение пропущенных значений нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Анализ выбросов в целевой переменной до обработки
print("=== АНАЛИЗ ВЫБРОСОВ В ЦЕЛЕВОЙ ПЕРЕМЕННОЙ ===")
print(f"Минимальная цена: {y_train.min():.0f}")
print(f"Максимальная цена: {y_train.max():.0f}")
print(f"Средняя цена: {y_train.mean():.0f}")
print(f"Медианная цена: {y_train.median():.0f}")

# Определение выбросов с помощью IQR
Q1 = y_train.quantile(0.25)
Q3 = y_train.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = y_train[(y_train < lower_bound) | (y_train > upper_bound)]
print(f"Количество выбросов (IQR метод): {len(outliers)}")
print(f"Доля выбросов: {len(outliers)/len(y_train)*100:.2f}%")

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Метод 1: Удаление выбросов по IQR
def remove_outliers_iqr(X, y):
    Q1 = y.quantile(0.25)
    Q3 = y.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    mask = (y >= lower_bound) & (y <= upper_bound)
    return X[mask], y[mask]

# Метод 2: Winsorization (ограничение выбросов)
def winsorize_target(y, limits=(0.05, 0.05)):
    return stats.mstats.winsorize(y, limits=limits)

mse_scores_train_outliers = []
mse_scores_val_outliers = []
mape_scores_train_outliers = []
mape_scores_val_outliers = []

print("\n=== С ОБРАБОТКОЙ ВЫБРОСОВ (IQR УДАЛЕНИЕ) ===")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    # Разделение данных
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Удаление выбросов только из тренировочной выборки
    X_train_clean, y_train_clean = remove_outliers_iqr(X_train_fold, y_train_fold)
    
    print(f"Fold {fold+1}: Исходный размер train: {len(X_train_fold)}, после удаления выбросов: {len(X_train_clean)}")
    
    # Масштабирование
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_clean)
    X_val_scaled = scaler.transform(X_val_fold)
    
    # Обучение модели
    model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
    model.fit(X_train_scaled, y_train_clean)
    
    # Предсказания
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    
    # Метрики
    mse_train = mean_squared_error(y_train_clean, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_clean, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_outliers.append(mse_train)
    mse_scores_val_outliers.append(mse_val)
    mape_scores_train_outliers.append(mape_train)
    mape_scores_val_outliers.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

results_outliers = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_outliers), np.mean(mse_scores_val_outliers)],
    'MSE_median': [np.median(mse_scores_train_outliers), np.median(mse_scores_val_outliers)],
    'MAPE_mean': [np.mean(mape_scores_train_outliers), np.mean(mape_scores_val_outliers)],
    'MAPE_median': [np.median(mape_scores_train_outliers), np.median(mape_scores_val_outliers)]
})

print("Результаты с обработкой выбросов:")
print(results_outliers)

# Обучение на полном датасете с удалением выбросов
X_train_clean_full, y_train_clean_full = remove_outliers_iqr(X_train, y_train)
scaler_full = StandardScaler()
X_train_scaled_full = scaler_full.fit_transform(X_train_clean_full)

final_model = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski', p=2)
final_model.fit(X_train_scaled_full, y_train_clean_full)

# Предсказания на полном train (без выбросов)
y_pred_full_train = final_model.predict(X_train_scaled_full)
mse_full_train = mean_squared_error(y_train_clean_full, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train_clean_full, y_pred_full_train)

print(f"\nMSE на полном train (без выбросов): {mse_full_train:.4f}")
print(f"MAPE на полном train (без выбросов): {mape_full_train:.4f}")
print(f"Размер данных после удаления выбросов: {len(X_train_clean_full)} из {len(X_train)}")

=== АНАЛИЗ ВЫБРОСОВ В ЦЕЛЕВОЙ ПЕРЕМЕННОЙ ===
Минимальная цена: 34900
Максимальная цена: 755000
Средняя цена: 180921
Медианная цена: 163000
Количество выбросов (IQR метод): 61
Доля выбросов: 4.18%

=== С ОБРАБОТКОЙ ВЫБРОСОВ (IQR УДАЛЕНИЕ) ===
Fold 1: Исходный размер train: 245, после удаления выбросов: 236
Fold 1: MSE_train=518010644.4217, MSE_val=2010778939.5969, MAPE_train=0.1048, MAPE_val=0.1435
Fold 2: Исходный размер train: 488, после удаления выбросов: 468
Fold 2: MSE_train=467122640.1678, MSE_val=2495326166.8227, MAPE_train=0.0954, MAPE_val=0.1554
Fold 3: Исходный размер train: 731, после удаления выбросов: 698
Fold 3: MSE_train=522383349.6424, MSE_val=1755628047.3032, MAPE_train=0.1025, MAPE_val=0.1228
Fold 4: Исходный размер train: 974, после удаления выбросов: 929
Fold 4: MSE_train=467055916.1624, MSE_val=2450972689.9067, MAPE_train=0.0997, MAPE_val=0.1255
Fold 5: Исходный размер train: 1217, после удаления выбросов: 1162
Fold 5: MSE_train=429710288.6806, MSE_val=1227481032.46

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

### Детальный анализ результатов

1. Качественное сравнение:
Обработка выбросов (П.7) vs Стандартный подход (П.2):

- MSE ухудшилась на 126.7% (с 877 млн до 1988 млн)

- MAPE ухудшился на 52.3% (с 8.77% до 13.35%)

- Значительное ухудшение на обеих метриках

2. Анализ проблемы:

Почему удаление выбросов не сработало:

__Потеря информативных данных:__

Удалено 61 объект (4.18% данных)

Эти "выбросы" могли содержать ценную информацию о реальных дорогих/дешевых домах

__Нарушение временной структуры:__

TimeSeriesSplit сохраняет временной порядок

Удаление объектов нарушает последовательность временных рядов

__Несогласованность train/validation:__

Выбросы удаляются только из train выборки

Validation выборка содержит выбросы, что создает дисбаланс

__Проблема определения выбросов:__

В данных о недвижимости экстремальные цены могут быть реальными, а не ошибками

Дорогие дома - это не выбросы, а закономерность рынка

3. Статистика обработки:
Исходный размер данных: 1460 объектов

После удаления выбросов: 1399 объектов

Удалено: 61 объект (4.18%)

Качество значительно ухудшилось, несмотря на небольшую потерю данных

Выводы

1. Лучший подход остается: П.2 (только численные признаки со StandardScaler)
Обоснование:

- Наилучшие результаты на валидационной выборке (MSE: 877 млн, MAPE: 8.77%)

- Сохранение всех данных, включая информативные "выбросы"

- Стабильное качество across всех фолдов

2. Итоговый рейтинг всех подходов:

1 место: П.2: StandardScaler для численных признаков

2 место: П.1: Без масштабирования

3 место: П.4: Комбинированный подход с категориальными

4 место: П.7 Обработка выбросов

5 место: П.6 Power Transform таргета

6 место: П.5 Power Transform признаков

7 место: П.3 Категориальные без масштабирования

3. Ключевые инсайты для kNN:

__Что НЕ работает в задаче предсказания цен на недвижимость:__

Удаление "выбросов" в ценах (они информативны)

Сложные нелинейные преобразования

OneHotEncoding для категориальных признаков

__Что РАБОТАЕТ:__

StandardScaler для численных признаков

Сохранение всех данных

Простота и минимализм в предобработке

### Заключение: 

Удаление выбросов в целевой переменной не улучшило, а значительно ухудшило качество модели kNN. Это подтверждает, что в данных о недвижимости экстремальные значения цен являются не шумом, а важной информацией о рынке. Для алгоритма kNN в этой задаче оптимальным остается простой подход со стандартным масштабированием численных признаков без удаления каких-либо данных. Сложные методы предобработки, включая обработку выбросов, не показали улучшения качества и часто приводили к ухудшению результатов.

# Пункт - 8 лучший дизайн + подбор гиперпараметров (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [47]:
# Используем лучший дизайн - только численные признаки с масштабированием
X_train = train.select_dtypes(include=[np.number]).drop('SalePrice', axis=1)
X_test = test.select_dtypes(include=[np.number])
y_train = train['SalePrice']

# Заполнение пропущенных значений нулями
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Схема валидации
cv = TimeSeriesSplit(n_splits=5, test_size=None, gap=0)

# Определение гиперпараметров для перебора
hyperparameters = [
    {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2},
    {'n_neighbors': 1, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2},
    {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2},
    {'n_neighbors': 7, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2},
    {'n_neighbors': 10, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2},
    {'n_neighbors': 5, 'weights': 'distance', 'metric': 'minkowski', 'p': 2},
    {'n_neighbors': 1, 'weights': 'distance', 'metric': 'minkowski', 'p': 2},
    {'n_neighbors': 3, 'weights': 'distance', 'metric': 'minkowski', 'p': 2},
    {'n_neighbors': 7, 'weights': 'distance', 'metric': 'minkowski', 'p': 2},
    {'n_neighbors': 10, 'weights': 'distance', 'metric': 'minkowski', 'p': 2}
]

# Перебор гиперпараметров
best_score = float('inf')
best_params = None
best_results = None

print("=== ПОДБОР ГИПЕРПАРАМЕТРОВ ===")

for params in hyperparameters:
    mse_scores_val = []
    mape_scores_val = []
    
    print(f"\nТестирование параметров: {params}")
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
        # Разделение данных
        X_train_fold = X_train.iloc[train_idx]
        X_val_fold = X_train.iloc[val_idx]
        y_train_fold = y_train.iloc[train_idx]
        y_val_fold = y_train.iloc[val_idx]
        
        # Масштабирование
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled = scaler.transform(X_val_fold)
        
        # Обучение модели с текущими параметрами
        model = KNeighborsRegressor(**params)
        model.fit(X_train_scaled, y_train_fold)
        
        # Предсказания на validation
        y_pred_val = model.predict(X_val_scaled)
        
        # Метрики
        mse_val = mean_squared_error(y_val_fold, y_pred_val)
        mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
        
        mse_scores_val.append(mse_val)
        mape_scores_val.append(mape_val)
    
    # Средние метрики по фолдам
    avg_mse = np.mean(mse_scores_val)
    avg_mape = np.mean(mape_scores_val)
    
    print(f"Средние метрики - MSE: {avg_mse:.4f}, MAPE: {avg_mape:.4f}")
    
    if avg_mse < best_score:
        best_score = avg_mse
        best_params = params
        best_results = {
            'mse_scores_val': mse_scores_val,
            'mape_scores_val': mape_scores_val,
            'avg_mse': avg_mse,
            'avg_mape': avg_mape
        }

print(f"\n=== ЛУЧШИЕ ПАРАМЕТРЫ ===")
print(f"Параметры: {best_params}")
print(f"Лучший MSE: {best_results['avg_mse']:.4f}")
print(f"Лучший MAPE: {best_results['avg_mape']:.4f}")

# Финальное обучение с лучшими параметрами на всех данных
print(f"\n=== ФИНАЛЬНАЯ МОДЕЛЬ С ЛУЧШИМИ ПАРАМЕТРАМИ ===")
mse_scores_train_final = []
mse_scores_val_final = []
mape_scores_train_final = []
mape_scores_val_final = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train)):
    # Разделение данных
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Масштабирование
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled = scaler.transform(X_val_fold)
    
    # Обучение модели с лучшими параметрами
    model = KNeighborsRegressor(**best_params)
    model.fit(X_train_scaled, y_train_fold)
    
    # Предсказания
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    
    # Метрики
    mse_train = mean_squared_error(y_train_fold, y_pred_train)
    mse_val = mean_squared_error(y_val_fold, y_pred_val)
    mape_train = mean_absolute_percentage_error(y_train_fold, y_pred_train)
    mape_val = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    
    mse_scores_train_final.append(mse_train)
    mse_scores_val_final.append(mse_val)
    mape_scores_train_final.append(mape_train)
    mape_scores_val_final.append(mape_val)
    
    print(f"Fold {fold+1}: MSE_train={mse_train:.4f}, MSE_val={mse_val:.4f}, "
          f"MAPE_train={mape_train:.4f}, MAPE_val={mape_val:.4f}")

results_final = pd.DataFrame({
    'Dataset': ['Train', 'Validation'],
    'MSE_mean': [np.mean(mse_scores_train_final), np.mean(mse_scores_val_final)],
    'MSE_median': [np.median(mse_scores_train_final), np.median(mse_scores_val_final)],
    'MAPE_mean': [np.mean(mape_scores_train_final), np.mean(mape_scores_val_final)],
    'MAPE_median': [np.median(mape_scores_train_final), np.median(mape_scores_val_final)]
})

print("\nФинальные результаты с лучшими гиперпараметрами:")
print(results_final)

# Обучение на полном датасете
scaler_final = StandardScaler()
X_train_scaled_final = scaler_final.fit_transform(X_train)

final_model = KNeighborsRegressor(**best_params)
final_model.fit(X_train_scaled_final, y_train)

y_pred_full_train = final_model.predict(X_train_scaled_final)
mse_full_train = mean_squared_error(y_train, y_pred_full_train)
mape_full_train = mean_absolute_percentage_error(y_train, y_pred_full_train)

print(f"\nMSE на полном train: {mse_full_train:.4f}")
print(f"MAPE на полном train: {mape_full_train:.4f}")
print(f"Лучшие параметры модели: {best_params}")

=== ПОДБОР ГИПЕРПАРАМЕТРОВ ===

Тестирование параметров: {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2}
Средние метрики - MSE: 1610819095.6890, MAPE: 0.1321

Тестирование параметров: {'n_neighbors': 1, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2}
Средние метрики - MSE: 2827053080.3440, MAPE: 0.1771

Тестирование параметров: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2}
Средние метрики - MSE: 1717313214.6339, MAPE: 0.1395

Тестирование параметров: {'n_neighbors': 7, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2}
Средние метрики - MSE: 1587430121.0126, MAPE: 0.1324

Тестирование параметров: {'n_neighbors': 10, 'weights': 'uniform', 'metric': 'minkowski', 'p': 2}
Средние метрики - MSE: 1651660208.0144, MAPE: 0.1315

Тестирование параметров: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'minkowski', 'p': 2}
Средние метрики - MSE: 1617037286.6555, MAPE: 0.1313

Тестирование параметров: {'n_neighbors': 1, 'weights': 'dist

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ